# ASSIGNMENT 7:

- Luis Felipe Acosta Zavaleta
- Alejandro Mosquera Ospina
- Raúl Emerson Amao Guerra
- Manuel Antonio Gil Soto Pisco
- Ruth Marina Chávez Pacheco

In [7]:
# We import the necessary libraries

!pip install pandas
!pip install geopandas
!pip install mapclassify

import pandas as pd
import os
import urllib.request, json, csv
import geopandas as gpd
import matplotlib.pyplot as plt
import mapclassify

ERROR: Could not find a version that satisfies the requirement geopandas (from versions: none)
ERROR: No matching distribution found for geopandas


   ---------------------------------------- 0.0/58.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/58.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/58.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/58.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/58.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/58.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/58.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/58.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/58.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/58.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/58.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/58.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/58.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/58.9 kB ? eta -:--:--
   -----------------

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\rosab\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\rosab\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\rosab\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\rosab\anaconda3\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\rosab\anaconda3\Lib\http\client.py", line 479, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\Users\rosab\anaconda3\Lib\socket.py", line 708, in readinto
    return self._sock.recv_into(b)
           ^

ModuleNotFoundError: No module named 'geopandas'

## 1. Import the data located at this link. It has information on people infected with dengue at the district level for 2015 to 2021.


In [ ]:
#We import the csv file

url = 'https://raw.githubusercontent.com/alexanderquispe/Diplomado_PUCP/main/_data/data_dengue_peru.csv'

df = pd.read_csv(url, low_memory = False)
print(df.head())



## 2. Generate ubigeo for Departments and Provinces taking the first two and four numbers.

In [ ]:
# We generate the requested columns

df['Ubigeo_Departamento'] = df['Ubigeo'].astype(str).str[:-4]  # This extracts all the numbers except the last four digits of the ubigeo column
df['Ubigeo_Provincia'] = df['Ubigeo'].astype(str).str[-4:]     #This extracts the last four digits of the ubigeo column

print(df[['Ubigeo', 'Ubigeo_Departamento', 'Ubigeo_Provincia']].head())

## 3. Use geopandas to plot the number of cases in 2021 by the district using a continuous legend. Do not forget to indicate the color of NA values. Use this shapefile.

In [ ]:
# Raw URL of the shapefile on GitHub
shp = r'https://github.com/alexanderquispe/Diplomado_PUCP/raw/main/_data/LIMITE_DISTRITAL_2020_INEI/INEI_LIMITE_DISTRITAL.shp'

# Read the shapefile using the raw URL
dist_shp = gpd.read_file(shp)

# Load the dengue data
url = 'https://raw.githubusercontent.com/alexanderquispe/Diplomado_PUCP/main/_data/data_dengue_peru.csv'
df_dengue = pd.read_csv(url, low_memory = False)

df_2021 = df_dengue[df_dengue['Año'] == 2021] #Filter for year 2021
df_2021.loc[:, 'Casos'] = pd.to_numeric(df_2021['Casos'], errors='coerce') #Convert the 'Casos' column to numeric, forcing non-numeric values to NaN
df_2021_grouped = df_2021.groupby('Ubigeo')['Casos'].sum().reset_index() #Group by Ubigeo of the district and sum the Casos column for each group

# Convert 'Casos' column to integers
df_2021_grouped['Casos'] = df_2021_grouped['Casos'].astype(int)

df_2021_grouped['Ubigeo'] = df_2021_grouped['Ubigeo'].astype(str)
dist_shp['UBIGEO'] = dist_shp['UBIGEO'].astype(str) #Ensure the 'Ubigeo' column is a string in both datasets

dist_shp1 = dist_shp.merge(df_2021_grouped, left_on='UBIGEO', right_on='Ubigeo', how='left') # Merge Dengue cases with the shapefile

#Plotting
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
dist_shp1.plot(column='Casos', cmap='OrRd', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True,
               missing_kwds={"color": "lightgrey", "label": "NaN"})

# Set plot title and labels
ax.set_title('Number of Dengue Cases by District (2021)', fontsize=15)

# Show the plot
plt.show()

## 4. Use geopandas to plot the number of cases in 2021 by the province using a continuous legend.

In [ ]:
# URL of the shapefile
shp_url = 'https://github.com/alexanderquispe/Diplomado_PUCP/raw/main/_data/LIMITE_DISTRITAL_2020_INEI/INEI_LIMITE_DISTRITAL.shp'

# Load the shapefile
df = gpd.read_file(shp_url)

# Load the dengue data
url = 'https://raw.githubusercontent.com/alexanderquispe/Diplomado_PUCP/main/_data/data_dengue_peru.csv' 
df_dengue = pd.read_csv(url, low_memory=False)

# Filter dengue data for the year 2021
df_2021 = df_dengue[df_dengue['Año'] == 2021].copy()  # Create a copy to avoid SettingWithCopyWarning

# Convert 'Casos' column to numeric, handling errors
df_2021.loc[:, 'Casos'] = pd.to_numeric(df_2021['Casos'], errors='coerce')

# Aggregate dengue cases by province
df_2021_grouped = df_2021.groupby('Ubigeo')['Casos'].sum().reset_index()
df_2021_grouped['Ubigeo'] = df_2021_grouped['Ubigeo'].astype(str)  # Ensure 'Ubigeo' is a string

# Prepare shapefile for merging
df['UBIGEO_PROV'] = df['UBIGEO'].astype(str).str[:4]  # Extract the first 4 digits for province-level aggregation

# Dissolve the shapefile by province level
prov_shp = df.dissolve(by='UBIGEO_PROV')

# Aggregate cases by province
cases_by_province = df_2021_grouped.groupby(df_2021_grouped['Ubigeo'].str[:4])['Casos'].sum().reset_index()
cases_by_province.columns = ['UBIGEO_PROV', 'Casos']

# Merge shapefile with case data
prov_shp = prov_shp.merge(cases_by_province, left_on='UBIGEO_PROV', right_on='UBIGEO_PROV', how='left')

# Convert 'Casos' to integer, handling NaN values
prov_shp['Casos'] = prov_shp['Casos'].fillna(0).astype(int)

# Plotting
fig, ax = plt.subplots(figsize=(10, 10))
prov_shp.plot(column='Casos', cmap='OrRd', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True,
              missing_kwds={"color": "lightgrey", "label": "No Data"})
ax.set_title('Number of Dengue Cases by Province (2021)', fontsize=15)
plt.show()
